LTP  
http://ltp.ai/docs/quickstart.html#id3   
https://github.com/HIT-SCIR/ltp  
synonyms    
https://www.52nlp.cn/synonyms-%E4%B8%AD%E6%96%87%E8%BF%91%E4%B9%89%E8%AF%8D%E5%B7%A5%E5%85%B7%E5%8C%85  
pkuseg   
https://gitee.com/mirrors/pkuseg/    
https://blog.csdn.net/ljz2016/article/details/103599548  
pypinyin  
https://pypinyin.readthedocs.io/zh_CN/master/index.html  
HanLP  
https://github.com/hankcs/HanLP  
https://hanlp.hankcs.com/docs/annotations/constituency/ctb.html

In [1]:
from ltp import LTP
# 加载基础模型
ltp = LTP(path='base.tgz') 
import synonyms
import pkuseg
import hanlp
from hanlp.components.mtl.multi_task_learning import MultiTaskLearning
from hanlp.components.mtl.tasks.tok.tag_tok import TaggingTokenization
import os
# 默认使用CPU
os.environ["CUDA_VISIBLE_DEVICES"]=""
HanLP: MultiTaskLearning = hanlp.load(hanlp.pretrained.mtl.CLOSE_TOK_POS_NER_SRL_DEP_SDP_CON_ELECTRA_BASE_ZH) # 世界最大中文语料库
tok: TaggingTokenization = HanLP['tok/fine']

smart_open library not found; falling back to local-filesystem-only
[jieba] default dict file path ..\data\vocab.txt
[jieba] default dict file path ..\data\vocab.txt
[jieba] load default dict ..\data\vocab.txt ...
[jieba] load default dict ..\data\vocab.txt ...
>> Synonyms load wordseg dict [C:\Users\USER\anaconda3\envs\nlp\lib\site-packages\synonyms\data\vocab.txt] ... 
>> Synonyms on loading stopwords [C:\Users\USER\anaconda3\envs\nlp\lib\site-packages\synonyms\data\stopwords.txt] ...
[Synonyms] on loading vectors [C:\Users\USER\anaconda3\envs\nlp\lib\site-packages\synonyms\data\words.vector.gz] ...


In [2]:
from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = 'all' #默认为'last'

In [3]:
import pandas as pd
import numpy as np
import re
from collections import ChainMap
from pypinyin import lazy_pinyin
from difflib import SequenceMatcher

In [4]:
data=pd.read_excel('理财产品属性.xlsx', index_col=0)

In [5]:
data.head(3)

,产品名称,产品代码,产品登记编码,本金及理财收益,巨额赎回,认购/申购上限,理财计划到期日,认购、申购及赎回方式,理财币种,产品类型,...,发行规模,认购起点,认购期,认购登记日,理财计划成立日,估值日,理财计划申购和赎回,费用,税款,投资策略
0,招商银行年年享5号理财计划,107205,C1030819000925,本理财计划不保证本金且不保证理财收益，收益随投资收益浮动。,投资者可于赎回日内向招商银行提出赎回申请，若理财计划每日净赎回额超过本理财计划上一工作日份额...,5000万元,2049年7月12日,个人投资者可通过招商银行各营业网点、招商银行财富账户、个人银行专业版、大众版或者招商银行认可...,人民币,固定收益类,...,1亿份-20亿份,1万份,2019年7月4日10:00-2019年7月11日17：00,2019年7月12日,2019年7月12日,本理财计划存续期内，每周三及开放日（或理财计划实际到期日）为估值日，如周三为非工作日，则估值...,本理财计划每12个月为一个投资周期，投资者可在每个投资周期结束日前7个工作日（含）至该投资周...,1.认/申购费：本理财计划不收取认/申购费。2.固定投资管理费：招商银行收取理财计划固定投资...,本理财计划运作过程中涉及的各纳税主体，其纳税义务按国家税收法律、法规执行。除法律法规特别要求...,"<table border=""1"" cellspacing=""0"" class=""dataf..."
1,招商银行年年享6号理财计划,107206,C1030819001083,本理财计划不保证本金且不保证理财收益，收益随投资收益浮动。,投资者可于赎回日内向招商银行提出赎回申请，若理财计划每日净赎回额超过本理财计划上一工作日份额...,5000万元,2049年8月13日,个人投资者可通过招商银行各营业网点、招商银行财富账户、个人银行专业版、大众版或者招商银行认可...,人民币,固定收益类,...,1亿份-30亿份,1万份,2019年8月5日10:00-2019年8月12日17：00,2019年8月13日,2019年8月13日,本理财计划存续期内，每周三及开放日（或理财计划实际到期日）为估值日，如周三为非工作日，则估值...,本理财计划每12个月为一个投资周期，投资者可在每个投资周期结束日前7个工作日（含）至该投资周...,1.认/申购费：本理财计划不收取认/申购费。2.固定投资管理费：招商银行收取理财计划固定投资...,本理财计划运作过程中涉及的各纳税主体，其纳税义务按国家税收法律、法规执行。除法律法规特别要求...,"<table border=""1"" cellspacing=""0"" class=""dataf..."
2,招商银行美元金葵一年定开1号理财计划,2101,C1030819001500,本理财计划不保障本金且不保证理财收益，收益随投资收益浮动。,投资者可于赎回日内向招商银行提出赎回申请，在理财计划存续期内的每个赎回日，若理财计划净赎回份...,1000万美元,2029年11月7日,个人投资者可通过招商银行各营业网点、招商银行财富账户、个人银行专业版、大众版或者招商银行认可...,美元,固定收益类,...,2000万份-3亿份,2000份,2019年10月30日10:00-2019年11月6日17:00,2019年11月7日,2019年11月7日,本理财计划存续期内，每周一及开放日（或理财计划实际到期日）为估值日，如周一为非交易日，则估值...,本理财计划每12个月为一个投资周期，投资者可在申购、赎回期提交本理财计划的申购、赎回申请。一...,1.认/申购费：本理财计划不收取认/申购费。2.固定投资管理费：招商银行收取理财计划固定投资...,本理财计划运作过程中涉及的各纳税主体，其纳税义务按国家税收法律、法规执行。除法律法规特别要求...,"<table border=""1"" cellspacing=""0"" class=""dataf..."


## 生成用户自定义词典

In [6]:
# 记录列属性进入专有名词库
columns = list(data.columns)
proper_noun = columns
# 将所有理财产品的特定名称和代码加入数据库
name = []
for col in ['产品名称', '产品代码', '产品登记编码']:
    name.append(list(data[col].map(lambda x: str(x)).values))
proper_noun = proper_noun + sum(name,[])
# 添加金融专有名词
with open("glossary.txt", "r", encoding='utf-8') as f:
    finance_noun = [i[:-1] for i in f.readlines()]
proper_noun = proper_noun + finance_noun
# 添加每个理财产品系列的专有名词
series_noun = ['零售青葵','青葵','美元金葵','金葵','季季开','年年享']
proper_noun = proper_noun + series_noun
# 重建属性字典
# 属性字典
attributes = {'本金及理财收益': ['本金', '保本'],
              '理财币种': ['货币类型', '理财货币', '投资币种'],
              '业绩比较基准': ['收益', '收益上限', '收益下限', '业绩比较基准的下限', '业绩比较基准的上限',
                         '理财收益', ],
              '发行规模': ['发行规模上限', '发行规模下限'],
              '产品类型': ['类型'],
              '认购起点': ['认购下限'],
              '理财计划申购和赎回': ['投资周期', '理财计划申购和赎回周期'],
              '理财计划到期日': ['到期日'],
              '募集方式': ['募集方法'],
              '投资策略': ['投资比例'],
              '认购期': ['认购期上限', '认购期下限', '认购期开始时间', '认购期结束时间'],
              '产品名称': ['理财产品']}
# 属性字典重构
attributes = dict([(i,k) for k in attributes.keys() for i in attributes[k]])
proper_noun = proper_noun + list(attributes.keys())
# 添加一些属性取值的专有名词
attribute_values = {}
for col in ['理财币种', '产品类型', '募集方式', '运作方式']:
    attribute_values[col] = np.unique(data[col].values).tolist()
proper_noun = proper_noun + list(attribute_values.values())
# 添加其他专有名词
proper_noun = proper_noun + ['理财产品']
# 计算最大前向分词窗口
max_windows = max([len(i) for i in proper_noun])

In [7]:
# 添加用户自定义词典
ltp.add_words(words=proper_noun, max_window=max_windows)

In [8]:
# 生成用户词典proper_noun.csv
pd.DataFrame(zip(proper_noun,['nz']*len(proper_noun))).to_csv('proper_noun.csv' ,header=0, index=0, sep=u'\t', encoding='utf_8_sig')

In [9]:
# 加载用户词典
seg = pkuseg.pkuseg(model_name = "default", user_dict = "./proper_noun.csv", postag = True)

In [10]:
# 添加用户自定义词典
tok.dict_combine = proper_noun

In [ ]:
# 加载停用词表
with open("scu_stopwords.txt", "r", encoding='utf-8') as f:
    stop_words = f.readlines()
stop_words = [sw[:-1] for sw in stop_words]
stop_words

## 构建相对应的词典

In [12]:
# 系列词典
series_dict = series_noun
# 名称词典
name_dict = [{columns[i]: name[i]} for i in range(3)]
# 属性词典
attri_dict = columns[3:] + list(attributes.keys()) + ['理财产品']

In [13]:
all_dict = {'系列':series_dict, '名称': name_dict, '属性': attri_dict}

In [14]:
# 加入系列名称的拼音以进行模糊匹配
all_dict['系列'] = [all_dict['系列'], [''.join(lazy_pinyin(n)) for n in all_dict['系列']]]

In [15]:
all_dict['系列']

[['零售青葵', '青葵', '美元金葵', '金葵', '季季开', '年年享'],
 ['lingshouqingkui',
  'qingkui',
  'meiyuanjinkui',
  'jinkui',
  'jijikai',
  'niannianxiang']]

In [16]:
all_dict['名称']

[{'产品名称': ['招商银行年年享5号理财计划',
   '招商银行年年享6号理财计划',
   '招商银行美元金葵一年定开1号理财计划',
   '招商银行美元金葵一年定开2号理财计划',
   '招商银行美元金葵一年定开3号理财计划',
   '招商银行美元金葵一年定开4号理财计划',
   '招商银行美元金葵一年定开6号理财计划',
   '招商银行美元金葵一年定开7号理财计划',
   '招商银行美元金葵季季开1号理财计划',
   '招商银行美元金葵季季开2号理财计划',
   '招商银行美元金葵季季开3号理财计划',
   '招商银行美元金葵季季开4号理财计划',
   '招商银行美元金葵季季开5号理财计划',
   '招商银行美元金葵季季开6号理财计划',
   '招商银行零售青葵系列一年半定开1号理财计划',
   '招商银行零售青葵系列一年半定开4号理财计划',
   '招商银行零售青葵系列一年半定开5号理财计划',
   '招商银行零售青葵系列一年定开1号理财计划',
   '招商银行零售青葵系列一年定开8号理财计划',
   '招商银行零售青葵系列一年定开9号理财计划',
   '招商银行零售青葵系列半年定开2号理财计划',
   '招商银行零售青葵系列半年定开3号理财计划',
   '招商银行零售青葵系列半年定开4号理财计划',
   '招商银行青葵系列一年半定开005号理财计划',
   '招商银行青葵系列一年半定开006号理财计划',
   '招商银行青葵系列一年半定开007号理财计划',
   '招商银行青葵系列一年半定开008号理财计划',
   '招商银行青葵系列一年半定开009号理财计划',
   '招商银行青葵系列一年半定开010号理财计划',
   '招商银行青葵系列一年半定开011号理财计划',
   '招商银行青葵系列一年半定开012号理财计划',
   '招商银行青葵系列一年定开004号理财计划',
   '招商银行青葵系列两年定开001号理财计划',
   '招商银行青葵系列两年定开002号理财计划',
   '招商银行青葵系列两年定开003号理财计划',
   '招商银行青葵系列两年定开004号理财计划',
   '招商银行青葵系列两年定开005号理财计划',
   '招商银行青葵系列两年定开006号理财计划',

In [17]:
all_dict['属性']

['本金及理财收益',
 '巨额赎回',
 '认购/申购上限',
 '理财计划到期日',
 '认购、申购及赎回方式',
 '理财币种',
 '产品类型',
 '募集方式',
 '运作方式',
 '业绩比较基准',
 '理财计划份额',
 '理财计划份额面值',
 '发行规模',
 '认购起点',
 '认购期',
 '认购登记日',
 '理财计划成立日',
 '估值日',
 '理财计划申购和赎回',
 '费用',
 '税款',
 '投资策略',
 '本金',
 '保本',
 '货币类型',
 '理财货币',
 '投资币种',
 '收益',
 '收益上限',
 '收益下限',
 '业绩比较基准的下限',
 '业绩比较基准的上限',
 '理财收益',
 '发行规模上限',
 '发行规模下限',
 '类型',
 '认购下限',
 '投资周期',
 '理财计划申购和赎回周期',
 '到期日',
 '募集方法',
 '投资比例',
 '认购期上限',
 '认购期下限',
 '认购期开始时间',
 '认购期结束时间',
 '理财产品',
 '理财产品']

## 问题解析

In [174]:
# 系列实体查找
def find_series(cut, pos, all_dict=all_dict):
    # 传入分词结果和对应的词性标注
    
    # 记录结果
    result = {}
    # 记录精确匹配结果
    series = {}
    # 记录模糊匹配结果
    series_dim = {}
    
    if '系列' in cut and ('理财计划' not in ''.join(cut) or '银行' not in ''.join(cut)):
        for idx, p in enumerate(pos):
            if cut[idx] == '系列':
                # 例如用户问：系列有哪些？
                if idx != 0:
                    if pos[idx-1] == 'nz':
                        # 进行分词适配
                        if len(cut[idx-1]) < 2 and idx >= 2:
                            cut[idx-1] = cut[idx-2] + cut[idx-1]

                        # 精确匹配
                        if cut[idx-1] in all_dict['系列'][0]:
                            series[cut[idx-1]] = cut[idx-1]

                        # 基于拼音的模糊匹配
                        elif ''.join(lazy_pinyin(cut[idx-1])) in all_dict['系列'][1]:
                            i = all_dict['系列'][1].index(''.join(lazy_pinyin(cut[idx-1])))
                            series_dim[cut[idx-1]] = all_dict['系列'][0][i]

        # 询问系列产品有哪些
        sent_return_all = ['系列产品','系列名称','系列列表', '所有系列']
        # 如果seires和series_dim都是空，则进行判断是否客户想查询的是所有系列名称
        if series == {} and series_dim == {}:
            points = [synonyms.compare(sent, ''.join(cut)) for sent in sent_return_all]
            if np.mean(points) >= 0.6:
                series_dim = {'所有系列':'所有系列'}

    result['series'] = series
    result['series_dim'] = series_dim
    return result
    

# 衡量两个实体的相似性
def similarity(a, b):
    # 引用ratio方法，返回序列相似性的度量
    points = SequenceMatcher(None, a, b).ratio()
    return points


# 名称实体查找
def find_obj(cut, pos, all_dict=all_dict):
    # 传入分词结果和对应的词性标注
    
    # 记录result
    result = {}
    # 记录error
    error = []
    # 记录精确匹配结果
    obj = {'产品名称':[], '产品代码':[], '产品登记编码':[]}
    # 记录模糊匹配结果
    obj_dim = {'产品名称':{}, '产品代码':{}, '产品登记编码':{}}
    
    # 精确匹配
    for idx, p in enumerate(pos):
        word = cut[idx]
        if p == 'nz' or p =='v':
            # 理财计划
            if '理财计划' in word:
                # 精确匹配
                if word in all_dict['名称'][0]['产品名称']:
                    obj['产品名称'].append(word)
            elif '代码' == word:
                # 精确匹配
                if cut[idx+1] in all_dict['名称'][1]['产品代码']:
                    obj['产品代码'].append(cut[idx+1])
            elif 'C' in word:
                # 精确匹配
                if word in all_dict['名称'][2]['产品登记编码']:
                     obj['产品登记编码'].append(word)
                
    
    # 对理财产品名称进行合并处理
    # 如果理财计划被单独分开了，则认为客户可能使用的是模糊匹配，故对理财产品名称进行合并
    if '理财计划' in cut:
        # 记录开头
        start = []
        # 记录结尾
        end = []   
        for idx, word in enumerate(cut):
            if '银行' in word and '理财计划' not in word:
                start.append(idx)
            elif '理财计划' in word and '银行' not in word:
                end.append(idx)
        if len(start) != len(end):
            error.append('传入理财产品名称格式错误：请以"xx银行"为开头，"理财计划"为结尾')
        else:
            names = [''.join(n) for n in [cut[s:e+1] for s,e in zip(start, end)]]
            recommend = [(name, n, similarity(n,name)) for n in all_dict['名称'][0]['产品名称'] for name in names]
            rec_dict = {}
            for rec in recommend:
                 rec_dict.setdefault(rec[0], []).append(rec[1:])
            for key in rec_dict.keys():
                recommend = sorted(rec_dict[key], key = lambda x:x[1], reverse=True)
                obj_dim['产品名称'][key] = [rec[0] for rec in recommend[:5]]
                # 直接以相似度最高的返回
                obj['产品名称'].append(recommend[0][0])
    
    # 对产品代码进行模糊匹配
    if '代码' in cut:
        # 记录代码位置
        code = []
        for idx, word in enumerate(cut):
            if '代码' == word and cut[idx+1] not in obj['产品代码'] and 'C' not in cut[idx+1]:
                code.append(idx+1)
        
        codes = [cut[c] for c in code]
        recommend = [(code, c, similarity(code,c)) for c in all_dict['名称'][1]['产品代码'] for code in codes]
        rec_dict = {}
        for rec in recommend:
             rec_dict.setdefault(rec[0], []).append(rec[1:])
        for key in rec_dict.keys():
            recommend = sorted(rec_dict[key], key = lambda x:x[1], reverse=True)
            obj_dim['产品代码'][key] = [rec[0] for rec in recommend[:5]]
            # 直接以相似度最高的返回
            obj['产品代码'].append(recommend[0][0])
                
    # 对产品登记编码进行模糊匹配
    num = np.sum(['C' in c for c in cut])
    if num != 0 and num > len(obj['产品登记编码']):
        # 记录需要模糊匹配的编码
        codes = []
        for idx, word in enumerate(cut):
            if 'C' in word and word not in obj['产品登记编码']:
                codes.append(word)
                
        recommend = [(code, c, similarity(code,c)) for c in all_dict['名称'][2]['产品登记编码'] for code in codes]
        rec_dict = {}
        for rec in recommend:
             rec_dict.setdefault(rec[0], []).append(rec[1:])
        for key in rec_dict.keys():
            recommend = sorted(rec_dict[key], key = lambda x:x[1], reverse=True)
            obj_dim['产品登记编码'][key] = [rec[0] for rec in recommend[:5]]
            # 直接以相似度最高的返回
            obj['产品登记编码'].append(recommend[0][0])
    
    result['obj'] = obj
    result['obj_dim'] = obj_dim
    result['Error'] = error
    return result


# 属性识别和归属
def find_attribute(string, all_dict=all_dict, seg=seg, HanLP=HanLP, stop_words=stop_words):
    
    # 记录结果
    result = {}
    # 记录错误
    error = []
    # 记录精确匹配实体的属性(属性，动词，值)
    attri_list = []
    # 记录模糊匹配实体的属性(属性，评分最高的前5个)
    attri_dim_dict = {}
    # 记录属性的开始和结束(属性，开始，结束)
    start_end = []
    # 记录实体属性间的关系(属性1，属性2，关系)
    attri_rel = []
    # 记录精确匹配的需要返回的属性
    attri_return = []
    # 记录模糊匹配的需要返回的属性
    attri_return_dim = {}
    # 要返回的属性的语义标注常见词汇
    return_value_ = ['多少','哪些','什么','那些']
    # 连接词
    conjunction = ['并且','并','和','或','而且',',','，']
    # ltp分词
    segment, hidden = ltp.seg([string])
    # ltp语义角色标注
    srl = ltp.srl(hidden)
    # 整合
    segment = pd.DataFrame(segment[0], columns=['word'])
    srl = srl[0]

    # 寻找语义角色标注中长度为1或大于2的标注（即对应着值或属性：值）
    for idx, s in enumerate(srl):
        attri_ = ''
        value_ = ''
        if len(s) == 1:
            # 如果为受事者，则向前寻找它的属性
            if s[0][0][:2] == 'A1':
                attri = s[0]
                value_ = segment.iloc[attri[1]: attri[2]+1].T
                attri_ = segment.iloc[idx-1, :].T
                attri_ = ''.join(attri_.values[0])
                value_ = ''.join(value_.values[0])
                
                 # 如果前一个是连词，也就是没有指明具体的属性，则认为是前一个属性
                if attri_ in conjunction:
                    # 则为前一个属性
                    attri_ = attri_list[-1][0]
                
                # 这里值即为属性，原本的值是产品或相对应的名称
                value = attri
                
        elif len(s) >= 2:
            attri = s[0]
            value = s[1]
            if len(s) >= 3:
                # 如果出现副词
                if 'ARGM-ADV' in [i[0] for i in s]:
                    i = [i[0] for i in s].index('ARGM-ADV')
                    if s[i-1][0][:2] == 'A0':
                        attri = s[i-1]
                        value = s[i]
                else:
                    # 取前后紧挨着的两个
                    for i in s:
                        if i[2] == idx - 1:
                            attri = i
                        elif i[1] == idx + 1:
                            value = i
            
            # 判断受事者和施事者顺序
            # 1、施事者在前，受事者在后
            if attri[0][:2] == 'A0' and value[0][:2] == 'A1':
                # 获取属性和值（中文）
                attri_ = segment.iloc[attri[1]: attri[2]+1].T
                if value[2] == list(segment.index)[-1]:
                    value_ = segment.iloc[value[1]:,].T
                else:
                    value_ = segment.iloc[value[1]: value[2]+1].T
                attri_ = ''.join(attri_.values[0])
                value_ = ''.join(value_.values[0])
                    
            # 2、施事者在后，受事者在前，则真正的施事者在受事者前
            elif attri[0][:2] == 'A1' and value[0][:2] == 'A0':
                value_ = segment.iloc[attri[1]: attri[2]+1].T
                attri_ = segment.iloc[idx-1, :].T
                attri_ = ''.join(attri_.values[0])
                value_ = ''.join(value_.values[0])
                
                # 如果前一个是连词，也就是没有指明具体的属性，则认为是前一个属性
                if attri_ in conjunction:
                    # 则为前一个属性
                    attri_ = attri_list[-1][0]
                
                # 这里值即为属性，原本的值是产品或相对应的名称
                value = attri
            
            # 3、类似语句：金葵系列中到期日比2030年4月3日早的所有产品的募集方式
            elif attri[0][:2] == 'A0' and value[0] == 'ARGM-ADV':
                attri_ = segment.iloc[attri[1]: attri[2]+1].T
                value_ = segment.iloc[value[1]: value[2]+1].T
                attri_ = ''.join(attri_.values[0])
                value_ = ''.join(value_.values[0])
                
            # 去除要返回的属性
            if value_ != '':
                if value_ in return_value_:
                    continue
                    
        if attri_ != '':
            # 精确匹配
            if attri_ in all_dict['属性']:
                attri_list.append([attri_, segment.iloc[idx,0], value_])
                if value[0] != 'ARGM-ADV':
                    start_end.append([attri_, attri[1], value[2]+1])
                else:
                    start_end.append([attri_, attri[1], idx+1])
            # 基于同义词的模糊匹配
            else:
                points = [(col, synonyms.compare(attri_, col)) for col in all_dict['属性']]
                recommend = sorted(points, key = lambda x:x[1], reverse=True)
                recommend = [list(rec) for rec in recommend]
                # 如果出现评分一样的行为，则再使用字符串匹配
                max_point = recommend[0][1]
                max_similarity = similarity(recommend[0][0], attri_)
                for rec in recommend[1:]:
                    if rec[1] == max_point and similarity(rec[0], attri_) > max_similarity:
                        max_similarity = similarity(rec[0], attri_)
                        recommend[0][0] = rec[0]
                        
                attri_list.append([recommend[0][0], segment.iloc[idx,0], value_])
                if value[0] != 'ARGM-ADV':
                    start_end.append([recommend[0][0], attri[1], value[2]+1])
                else:
                    start_end.append([recommend[0][0], attri[1], idx+1])
                attri_dim_dict[attri_] = [rec[0] for rec in recommend[:5]]  

    # 获取两个邻近属性之间的关系
    for idx, attri in enumerate(attri_list):
        # 如果不是最后一个
        if idx <= len(attri_list) - 2:
            key1 = attri[0]
            key2 = attri_list[idx+1][0]
            start = start_end[idx][2]
            end = start_end[idx+1][1]
            
            if start_end[idx][1] == start_end[idx+1][1] and start_end[idx][2] < start_end[idx+1][2]:
                # 需要对位置进行修正
                value1 = ''.join(list(segment.loc[start_end[idx+1][1]:start_end[idx+1][2],'word'].values))
                # 找到第一个连词并将其分离
                cut, pos = list(zip(*seg.cut(value1)))
                for i, p in enumerate(pos):
                    if p == 'c':
                        value1 = ''.join(cut[:i])
                        rel = cut[i]
                        break
                attri_rel.append([key1, key2, rel])
            elif start > end:
                # 需要对value1进行修正
                value1 = attri[2]
                # 找到第一个连词并将其分离
                cut, pos = list(zip(*seg.cut(value1)))
                for i, p in enumerate(pos):
                    if p == 'c':
                        value1 = ''.join(cut[:i])
                        rel = cut[i]
                        break
                attri_list[idx][2] = value1
                attri_rel.append([key1, key2, rel])             
            else:
                conjunct = segment.iloc[start:end, 0].T.values[0]
                for c in conjunct:
                    if c in conjunction:
                        attri_rel.append([key1, key2, conjunct])
    
    # 句子中逗号和顿号的转换
    new_string = string.replace(',','和').replace('，','和').replace('、','和')
    
    # 去除停用词(保留关系词)
    segment = HanLP(new_string)['tok/fine']
    new_string = ''
    for s in segment:
        if s in conjunction:
            new_string = new_string + s
        elif s not in stop_words:
            new_string = new_string + s
    
    # 识别需要返回的属性
    tree = HanLP(new_string)['con']
    words = None
    location = tree[-1][-1]
    while True:
        # 尝试识别最后的NP
        if not isinstance(location, str):
            if location.label() == 'NP' :
                if isinstance(location[-1], str):
                    words = location.pos()
                    break
                elif location[-1].label() != 'NP':
                    words = location.pos()
                    break
                else:
                    location = location[-1]
            else:
                location = location[-1]
        else:
            break

    if words is None:
        location = tree[0]
        while True:
            # 尝试识别最前面的的DNP
            if not isinstance(location, str):
                if location.label() == 'DNP' :
                    words = location.leaves()
                    new_string = new_string.replace(''.join(words),'')
                    words = HanLP(new_string)['con'].pos()
                    break
                else:
                    location = location[-1]
            else:
                error.append('可能没有要返回的属性！')
                break
    
    if words is not None:
        # 记录需要删除的idx
        del_idx = []
        # 对words进行清理(去除系列或名称)
        for i, w in enumerate(words):
            if w[0] in all_dict['系列'][0]:
                del_idx.append([i,i+1])
            elif w[0] in (all_dict['名称'][0]['产品名称'] + all_dict['名称'][2]['产品登记编码']):
                del_idx.append([i])
            elif w[0] in all_dict['名称'][1]['产品代码']:
                del_idx.append([i-1,i])

        words =  [words[i] for i in range(len(words)) if i not in sum(del_idx,[])]

        # 识别words中的连词作为合并属性的依据
        word = ''
        return_attri = []
        for w in words:
            if w == words[-1]:
                word = word + w[0]
                return_attri.append(word)    
            elif w[1] == 'CC':
                return_attri.append(word)
                word = ''
            elif w[0] not in ['产品','理财产品','的']:
                word = word + w[0]
        
        new_attri = all_dict['属性'] + [list(i.keys())[0] for i in all_dict['名称']]
        for attri_ in return_attri:
            
            if attri_ != '':
                
                # 精准匹配
                if attri_ in  new_attri:
                    attri_return.append(attri_)

                # 基于同义词的模糊匹配
                else:
                    points = [(col, synonyms.compare(attri_, col)) for col in  new_attri]
                    recommend = sorted(points, key = lambda x:x[1], reverse=True)
                    recommend = [list(rec) for rec in recommend]
                    # 如果出现评分一样的行为，则再使用字符串匹配
                    max_point = recommend[0][1]
                    max_similarity = similarity(recommend[0][0], attri_)
                    for rec in recommend[1:]:
                        if rec[1] == max_point and similarity(rec[0], attri_) > max_similarity:
                            max_similarity = similarity(rec[0], attri_)
                            recommend[0][0] = rec[0]
                    attri_return.append(recommend[0][0])
                    attri_return_dim[attri_] = [rec[0] for rec in recommend[:5]]
        
    print(attri_return)
    result['attri'] = attri_list
    result['attri_dim'] = attri_dim_dict 
    result['start_end'] = start_end
    result['attri_rel'] = attri_rel
    result['attri_return'] = attri_return
    result['attri_return_dim'] = attri_return_dim
    result['error'] = error
    return result
    

# 问题解析主函数
def main_handle(string, seg=seg, HanLP=HanLP, all_dict=all_dict, stop_words=stop_words):
    # 记录错误
    Error = []
    # 需要返回给用户确认的
    re_confirm = {}
    # 删除最后的标点
    if string[-1] in ['.','。',',','，','？','?']:
        string = string[:-1]
    # 分词+词性标注
    cut, pos = list(zip(*seg.cut(string)))
    cut = list(cut)
    
    # 进行系列匹配
    # 提问标准1：用户必须为需要查找的所有系列后面都加上“系列”
    # * 正确：请告诉我青葵系列和金葵系列所有产品的名称
    # * 错误：请告诉我青葵和金葵系列所有产品的名称
    # 提问标准2：不能同时对系列和单一的理财产品进行查询

    serieses = find_series(cut, pos)
    series = serieses['series']
    series_dim = serieses['series_dim']
   
    # 合并二者成为需要查询的系列表
    if '所有系列' in series_dim:
        # 返回给查询
        return {'neo4j': [[{'名称':'所有系列'}],[]]}
    elif series != {} or series_dim != {}:
        series_all = list(series.values()) + list(series_dim.values())
        if cut.count('系列') != len(series_all):
            Error.append(['只识别出了{}共{}个系列，请确认未识别的系列的名称是否写对！'.format(series, len(series))])
    else:
        if '系列' in cut and '理财计划' not in ''.join(cut):
            Error = '抱歉，数据库中没有相关系列产品，数据库中目前只有以下系列产品:'
            return {'neo4j': [[{'名称':'所有系列'}],[]], 'Error': Error}
    
    # 规范系列名称
    if series_dim != {}:
        for dim in list(series_dim.keys()):
            string = string.replace(dim, series_dim[dim])
    
    # 进行实体匹配（名称、产品代码和产品登记编码）
    # 提问标准3：用户在对名称进行提问时必须要以“xx银行”开头“理财计划”结尾
    # 提问标准4：用户在对产品代码进行提问时必须要以“代码”开头，并且之后直接接代码
    # 提问标准5：用户在对产品登记编码进行提问时必须要以“C”开头
    objs = find_obj(cut, pos)
    obj = objs['obj']
    obj_dim = objs['obj_dim']
    re_confirm['理财产品实体确认'] = objs['obj_dim']
    Error.append(objs['Error'])
    
    # 规范实体名称
    for key in obj_dim.keys():
        if obj_dim[key] != {}:
            for k in list(obj_dim[key].keys()):
                string = string.replace(k, obj_dim[key][k][0])
    
    # 进行实体的属性匹配
    attris = find_attribute(string) 

    
    return serieses, objs, attris, Error

In [175]:
result = main_handle("到期日早于2015年的产品名称、产品登记编码和到期日")

['产品名称', '产品登记编码', '到期日']


In [176]:
result

({'series': {}, 'series_dim': {}},
 {'obj': {'产品名称': [], '产品代码': [], '产品登记编码': []},
  'obj_dim': {'产品名称': {}, '产品代码': {}, '产品登记编码': {}},
  'Error': []},
 {'attri': [['到期日', '早', '于2015年']],
  'attri_dim': {},
  'start_end': [['到期日', 0, 4]],
  'attri_rel': [],
  'attri_return': ['产品名称', '产品登记编码', '到期日'],
  'attri_return_dim': {},
  'error': []},
 [[]])

In [177]:
x = result[0]
for r in result:
    if r != x:
        z = ChainMap(x,r)
        x = z

In [178]:
x

ChainMap(ChainMap(ChainMap({'series': {}, 'series_dim': {}}, {'obj': {'产品名称': [], '产品代码': [], '产品登记编码': []}, 'obj_dim': {'产品名称': {}, '产品代码': {}, '产品登记编码': {}}, 'Error': []}), {'attri': [['到期日', '早', '于2015年']], 'attri_dim': {}, 'start_end': [['到期日', 0, 4]], 'attri_rel': [], 'attri_return': ['产品名称', '产品登记编码', '到期日'], 'attri_return_dim': {}, 'error': []}), [[]])

In [179]:
series = {'series': list(x['series'].values()) + list(x['series_dim'].values())}

In [180]:
series

{'series': []}

In [181]:
obj = x['obj']

In [182]:
obj

{'产品名称': [], '产品代码': [], '产品登记编码': []}

In [183]:
obj = dict(obj, **series)

In [184]:
obj

{'产品名称': [], '产品代码': [], '产品登记编码': [], 'series': []}

In [185]:
obj['Name'] = obj.pop('产品名称')
obj['Code'] = obj.pop('产品代码')
obj['OuterCode'] = obj.pop('产品登记编码')

In [186]:
obj

{'series': [], 'Name': [], 'Code': [], 'OuterCode': []}

In [187]:
attributes

{'本金': '本金及理财收益',
 '保本': '本金及理财收益',
 '货币类型': '理财币种',
 '理财货币': '理财币种',
 '投资币种': '理财币种',
 '收益': '业绩比较基准',
 '收益上限': '业绩比较基准',
 '收益下限': '业绩比较基准',
 '业绩比较基准的下限': '业绩比较基准',
 '业绩比较基准的上限': '业绩比较基准',
 '理财收益': '业绩比较基准',
 '发行规模上限': '发行规模',
 '发行规模下限': '发行规模',
 '类型': '产品类型',
 '认购下限': '认购起点',
 '投资周期': '理财计划申购和赎回',
 '理财计划申购和赎回周期': '理财计划申购和赎回',
 '到期日': '理财计划到期日',
 '募集方法': '募集方式',
 '投资比例': '投资策略',
 '认购期上限': '认购期',
 '认购期下限': '认购期',
 '认购期开始时间': '认购期',
 '认购期结束时间': '认购期',
 '理财产品': '产品名称'}

In [188]:
return_ = x['attri_return']

In [189]:
return_

['产品名称', '产品登记编码', '到期日']

In [190]:
for idx, r in enumerate(return_):
    if r in attributes:
        if '上限' in r or '下限' in r:
            return_[idx] = attributes[r] + '-' + r[-2:]
        else:
            return_[idx] = attributes[r]

In [109]:
return_

['募集方式']

In [89]:
attri = x['attri']

In [90]:
attri

[['到期日', '早', '比2030年4月3日'], ['投资币种', '为', '美元']]

In [91]:
attri_all = {}

In [92]:
Error = []

In [93]:
for a in attri:
    
    # 获取attribute中的属性、动词和值
    name = a[0]
    verb = a[1]
    value = a[2]
    
    # 获取attribute对应的名称
    if name in set(attributes.values()):
        real_attribute = name
    else:
        real_attribute = attributes[name]
    
    # 对于日期类型的查询，有关的判断
    time_greater_judge = ['晚','后']
    time_less_judge = ['早','先','前']
    # 对于数值类型的查询，有关的判断
    number_greater_judge = ['大','高','超']
    number_less_judge = ['小','低']
    
    
    # 记录特殊属性
    special_attri = ''
    # 记录符号
    symbol = ''
    # 允许比较大小的属性
    compared_attri = ['认购/申购上限', '理财计划到期日', '业绩比较基准', '发行规模', '认购起点', '认购期',
                      '认购登记日', '理财计划成立日']
    # 可以比较的属性中的数值型属性
    number_attri = ['认购/申购上限', '业绩比较基准', '发行规模', '认购起点']
    # 可以比较的属性中的日期属性
    date_attri = ['理财计划到期日', '认购期', '认购登记日', '理财计划成立日']
    # 可以有多个离散取值的属性
    multi_attri = ['理财币种', '产品类型', '募集方式', '运作方式']
    # 存储结果
    alist = attri_all.setdefault(real_attribute, [])
    
    # 如果属性中出现五个关键词，则提取
    if '上限' in name or '下限' in name or '周期' in name:
        special_attri = name[-2:]
    elif '开始时间' in name:
        special_attri = '下限'
    elif '结束时间' in name:
        special_attri = '上限'
        
    
    
    # 如果属性中出现的是日期的属性
    if real_attribute in date_attri:

        # 使用正则表达式匹配日期
        pattern = re.compile(r'\d{4}[年\-/]*\d{0,2}[月\-/]*\d{0,2}[日]*')
        dates = pattern.findall(value)
        stand_value = []
        # 规范日期格式
        for date in dates:
            if '年' in date:
                year = date[:date.find('年')]
                month = date[date.find('年')+1:date.find('月')] if date.find('月') != -1 else '01'
                day = date[date.find('月')+1:date.find('日')] if date.find('日') != -1 else '01'
                stand_value.append(year+month+day)
            else:
                try:
                    stand_value.append(pd.to_datetime(date).strftime('%Y%m%d'))
                except:
                    Error.append('请按照以下日期格式重新输入正确的日期！\n格式一：2021年1月1日\n格式二：2021-01-01\n格式三：2021/01/01')

        # 如果有多个日期，则提取它们之间的关系(两种：一种是在..和..间， 另一种是为..和(或)..)
        if len(stand_value) >= 2:
            if '间' in value and len(dates) == 2:
                alist.append(special_attri+'-'+'大于'+'-'+stand_value[0])
                alist.append('并且')
                alist.append(special_attri+'-'+'小于'+'-'+stand_value[1])
            else:
                if '或' in value:
                    for date in stand_value:
                        alist.append(special_attri+'-'+'等于'+'-'+date)
                        if date != stand_value[-1]:
                            alist.append('或')
                else:
                    for date in stand_value:
                        alist.append(special_attri+'-'+'等于'+'-'+date)
                        if date != stand_value[-1]:
                            alist.append('并且')
                        
        # 如果只存在一个日期，则需要根据动词和value判断符号
        else:
            has_belong = False
            
            for j in time_greater_judge:
                if j in verb or j in value:
                    alist.append(special_attri+'-'+'大于'+'-'+stand_value[0])
                    has_belong = True
                    
            if not has_belong:
                for j in time_less_judge:
                    if j in verb or j in value:
                        alist.append(special_attri+'-'+'小于'+'-'+stand_value[0])
                        has_belong = True
            
            if not has_belong:
                alist.append(special_attri+'-'+'等于'+'-'+stand_value[0])
                
    
    # 如果属性中出现的是数值型的属性
    elif real_attribute in number_attri:
        
        # 使用正则表达式匹配数值
        pattern = re.compile(r'[0-9%.]+')
        datas = pattern.findall(value)
        stand_value = []
        # 规范数字格式
        for data in datas:
            if '%' in data:
                stand_value.append(data[:-1])
            elif '.' in data and float(data) < 1:
                stand_value.append(float(data) * 100)
            else:
                stand_value.append(int(float(data)))
        
        # 如果有多个数值，则提取它们之间的关系(两种：一种是在..和..间， 另一种是为..和(或)..)
        if len(stand_value) >= 2:
            if '间' in value and len(dates) == 2:
                alist.append(special_attri+'-'+'大于'+'-'+stand_value[0])
                alist.append('并且')
                alist.append(special_attri+'-'+'小于'+'-'+stand_value[1])
            else:
                if '或' in value:
                    for data in stand_value:
                        alist.append(special_attri+'-'+'等于'+'-'+data)
                        if data != stand_value[-1]:
                            alist.append('或')
                else:
                    for data in stand_value:
                        alist.append(special_attri+'-'+'等于'+'-'+data)
                        if data != stand_value[-1]:
                            alist.append('并且')
                        
        # 如果只存在一个数值，则需要根据动词和value判断符号
        else:
            has_belong = False
            
            for j in number_greater_judge:
                if j in verb or j in value:
                    alist.append(special_attri+'-'+'大于'+'-'+stand_value[0])
                    has_belong = True
                    
            if not has_belong:
                for j in number_less_judge:
                    if j in verb or j in value:
                        alist.append(special_attri+'-'+'小于'+'-'+stand_value[0])
                        has_belong = True
            
            if not has_belong:
                alist.append(special_attri+'-'+'等于'+'-'+stand_value[0])
                
    
    # 如果出现的是有多个离散取值的属性
    elif real_attribute in multi_attri:
        # 只需分离出符号和连接的关系即可
        cut, pos = list(zip(*seg.cut(value))) 
        i = 0
        stand_value = []
        
        # 合并属性
        for idx, p in enumerate(pos):
            if (p == 'w' or p == 'c') and idx != len(pos)-1 :
                stand_value.append(''.join(cut[i:idx]))
                i = idx+1
            elif idx == len(pos)-1:
                stand_value.append(''.join(cut[i:]))

        # 对专有属性的一个匹配，若得分超过75%，则认为顾客查询的是这个属性
        for idx, noun in enumerate(stand_value):
            if noun not in ['美金','欧元','日元','韩元','卢布']:
                points = [{v: synonyms.compare(noun, v)} for v in attribute_values[real_attribute]]
                points = sorted(points, key = lambda x:list(x.values())[0], reverse=True)
                if list(points[0].values())[0] > 0.75:
                    stand_value[idx] = list(points[0].keys())[0]
                else:
                    stand_value[idx] = noun
                    
            else:
                if noun == '美金':
                    stand_value[idx] = '美元'
                else:
                    stand_value[idx] = noun
                
        
        # 提取它们之间的关系
        if '或' in value:
            for data in stand_value:
                alist.append('等于'+'-'+data)
                if data != stand_value[-1]:
                    alist.append('或')
        else:
            for data in stand_value:
                alist.append('等于'+'-'+data)
                if data != stand_value[-1]:
                    alist.append('并且')

    else:
        Error.append('您要查询的属性暂时还不支持用于限定类查询！')
    

In [94]:
attri_all

{'理财计划到期日': ['-小于-203043'], '理财币种': ['等于-美元']}

In [74]:
seg.cut('固定收益，浮动收益或稳定收益类')

[('固定收益', 'nz'),
 ('，', 'w'),
 ('浮动', 'v'),
 ('收益', 'nz'),
 ('或', 'c'),
 ('稳定', 'a'),
 ('收益', 'nz'),
 ('类', 'n')]

In [83]:
synonyms.compare('稳定收益类','固定收益类型')

0.926

In [137]:
seg1, hidden = ltp.seg(["金葵系列中，发行规模下限比100000000大并且上限比2000000000小的理财产品的到期日和理财产品登记日"])
srl = ltp.srl(hidden)

In [138]:
srl[0]

[[],
 [],
 [],
 [],
 [],
 [],
 [],
 [('A0', 4, 4), ('ARGM-ADV', 5, 6)],
 [],
 [],
 [],
 [],
 [('A0', 9, 9), ('ARGM-ADV', 10, 11)],
 [],
 [],
 [],
 [],
 [],
 [],
 []]

In [139]:
pd.DataFrame(seg1[0])

,0
0,金葵
1,系列
2,中
3,，
4,发行规模下限
5,比
6,100000000
7,大
8,并且
9,上限


In [459]:
sdp = ltp.sdp(hidden,True)
sdp

[[(1, 2, 'mDEPD'),
  (3, 2, 'DATV'),
  (4, 9, 'mRELA'),
  (5, 6, 'FEAT'),
  (6, 9, 'eCOO'),
  (7, 9, 'mRELA'),
  (8, 9, 'FEAT'),
  (10, 9, 'mDEPD'),
  (11, 9, 'mPUNC'),
  (12, 13, 'EXP'),
  (14, 15, 'FEAT'),
  (14, 16, 'FEAT'),
  (15, 16, 'FEAT'),
  (16, 13, 'TIME'),
  (17, 13, 'mDEPD'),
  (18, 20, 'mRELA'),
  (21, 20, 'LINK'),
  (22, 23, 'mRELA'),
  (23, 21, 'eCOO'),
  (24, 20, 'mDEPD'),
  (26, 20, 'mDEPD'),
  (28, 2, 'dCONT'),
  (29, 28, 'LINK')]]

In [460]:
list(zip(seg1[0],list(range(len(seg1[0])+1))[1:]))

[('请', 1),
 ('告诉', 2),
 ('我', 3),
 ('在', 4),
 ('青葵', 5),
 ('系列', 6),
 ('和', 7),
 ('金葵', 8),
 ('系列', 9),
 ('中', 10),
 ('，', 11),
 ('到期日', 12),
 ('在', 13),
 ('2021年', 14),
 ('6月', 15),
 ('20日', 16),
 ('之前', 17),
 ('并且', 18),
 ('理财币种', 19),
 ('是', 20),
 ('美元', 21),
 ('或', 22),
 ('人民币', 23),
 ('的', 24),
 ('理财产品', 25),
 ('的', 26),
 ('投资策略', 27),
 ('是', 28),
 ('什么', 29)]

In [ ]:
HanLP(seg.cut('金匮系列中到期日比2030年2月早的产品的募集方式'), tasks='dep')

In [24]:
HanLP(list(list(zip(*seg.cut('募集方式为公募并且理财货币为人民币或美元的理财产品的产品名称和本金及理财收益.')))[0]),\
      tasks=['pos', 'dep'], skip_tasks='tok*')

{'pos/ctb': ['NN',
  'VC',
  'VV',
  'CC',
  'NN',
  'VC',
  'NN',
  'CC',
  'M',
  'DEG',
  'NN',
  'DEG',
  'NN',
  'NN',
  'CC',
  'NN'],
 'dep': [(2, 'top'),
  (0, 'root'),
  (2, 'attr'),
  (2, 'cc'),
  (6, 'top'),
  (2, 'conj'),
  (9, 'conj'),
  (9, 'cc'),
  (6, 'attr'),
  (6, 'cpm'),
  (13, 'assmod'),
  (11, 'assm'),
  (14, 'nn'),
  (16, 'conj'),
  (16, 'cc'),
  (2, 'attr')],
 'tok': ['募集方式',
  '为',
  '公募',
  '并且',
  '理财货币',
  '为',
  '人民币',
  '或',
  '美元',
  '的',
  '理财产品',
  '的',
  '产品',
  '名称',
  '和',
  '本金及理财收益']}

In [18]:
import hanlp
from hanlp.components.mtl.multi_task_learning import MultiTaskLearning
from hanlp.components.mtl.tasks.tok.tag_tok import TaggingTokenization

HanLP: MultiTaskLearning = hanlp.load(hanlp.pretrained.mtl.CLOSE_TOK_POS_NER_SRL_DEP_SDP_CON_ELECTRA_BASE_ZH) # 世界最大中文语料库
tok: TaggingTokenization = HanLP['tok/fine']

# tok.dict_force = tok.dict_combine = None
# print(f'不挂词典:\n{HanLP("商品和服务行业")["tok/fine"]}')
#
# tok.dict_force = {'和服', '服务行业'}
# print(f'强制模式:\n{HanLP("商品和服务行业")["tok/fine"]}')  # 慎用，详见《自然语言处理入门》第二章
#
# tok.dict_force = {'和服务': ['和', '服务']}
# print(f'强制校正:\n{HanLP("正向匹配商品和服务、任何和服务必按上述切分")["tok/fine"]}')

tok.dict_force = None
tok.dict_combine = ['招商银行年年享6号理财计划']
print(f'合并模式:\n{HanLP("招商银行年年享6号理财计划的范围有哪些")["tok/fine"]}')

合并模式:
['招商银行年年享6号理财计划', '的', '范围', '有', '哪些']


In [400]:
HanLP("金葵系列中到期日比2030年4月3日早，投资币种为美元的所有产品的募集方式和投资比例")['tok/fine']

['金葵',
 '系列',
 '中',
 '到期日',
 '比',
 '2030年',
 '4月',
 '3日',
 '早',
 '，',
 '投资币种',
 '为',
 '美元',
 '的',
 '所有',
 '产品',
 '的',
 '募集方式',
 '和',
 '投资比例']

In [168]:
a = HanLP("到期日早于2015年的产品名称、产品登记编码和到期日")
a.pretty_print()

In [48]:
a['con'][-1][-1]

['NP', [['NP', [['NN', ['募集方式']]]], ['CC', ['和']], ['NP', [['NN', ['产品名称']]]]]]

In [33]:
a['con'][-1][-1].label()

'NP'

In [489]:
a['con'][-2].label()

'CC'

In [360]:
location

['NP', [['NN', ['募集方式']], ['CC', ['和']], ['NN', ['投资比例']]]]

In [500]:
a['con'][0].leaves()

['青葵', '系列', '产品', '的']

In [131]:
s1=[{'理财计划到期日': ['上限', '小于', '2030年4月3日']}, {'产品名称': '金葵'},{'理财计划': ['2030年4月3日']}]
ss=[{'理财计划到期日': ['小于', '2030年4月3日']}]
s2=[{'认购起点': ['大于', '2000份']}]
def change_symbol(items):
    output=[]
    for item in items:
        a=str(item.keys())[12:-3]
        b=str(item.values())[14:-3]
        item_list=",".join(str(i) for i in item.values())
        if ('大于'in item_list) or ('小于' in item_list):
            ste = re.sub("[A-Za-z\!\%\[\]\,\。\-\_\']", "", item_list)
            ste = [re.sub("[\ ]", "-", ste)]
            output.append({a:ste})
        else:
            output.append({a:b})
    print(output)
        
        
change_symbol(s1)
change_symbol(s2)
change_symbol(ss)

[{'理财计划到期日': ['上限-小于-2030年4月3日']}, {'产品名称': '金葵'}, {'理财计划': "'2030年4月3日'"}]
[{'认购起点': ['大于-2000份']}]
[{'理财计划到期日': ['小于-2030年4月3日']}]


In [132]:
str(['a','b'])

"['a', 'b']"

In [133]:
'asd'.split('-')

['asd']

In [134]:
a = [1,3,2]

In [135]:
a[-1] =2

In [136]:
a

[1, 3, 2]

In [137]:
a[-1] =10

In [138]:
a

[1, 3, 10]